# Analyzing LFQ data of cell culture protein extracts

## Preparation for Analysis

### Load Data

In [ ]:
source("../classes/data_loader.R")
source("../utils/data.R")

### Local Settings

In [ ]:
create_directory <- function(path) {
  if (dir.exists(path)) {
    warning(sprintf("Deleting %s", path))
    unlink(path, recursive = TRUE)
  }

  warning(sprintf("Creating %s", path))
  dir.create(path, recursive = TRUE)
  file.create(file.path(path, ".gitkeep"))
}

In [ ]:
technical_analysis_directory <- "../results/lfq/technical-analysis/"
biological_analysis_directory <- "../results/lfq/biological-analysis/"

create_directory(technical_analysis_directory)
create_directory(biological_analysis_directory)

## Technical Analysis

### Features

In [ ]:
source("../classes/feature_identification.R")

feature_identification <- new("FeatureIdentification",
  a_name = "Sequest HT",
  ms2_data_frame_a = data@lfqSequestHT_ms2,
  psm_data_frame_a = data@lfqSequestHT_psms,
  protein_data_frame_a = data@lfqSequestHT_proteins,
  protein_data_frame_a_complete = data@lfqSequestHT_proteins_complete,
  b_name = "CHIMERYS",
  ms2_data_frame_b = data@lfqChimerys_ms2,
  psm_data_frame_b = data@lfqChimerys_psms,
  protein_data_frame_b = data@lfqChimerys_proteins,
  protein_data_frame_b_complete = data@lfqChimerys_proteins_complete
)

In [ ]:
plotFeatures(feature_identification, directory = technical_analysis_directory)

In [ ]:
plotIdRate(feature_identification, directory = technical_analysis_directory)

In [ ]:
plotIdRate(feature_identification, plot_mode = "relative", directory = technical_analysis_directory)

### Mass Accuracy

In [ ]:
source("../classes/mass_accuracy.R")

mass_accuracy <- new("MassAccuracy", psm_data_frame = data@lfqSequestHT_psms)

In [ ]:
plotViolin(mass_accuracy, directory = technical_analysis_directory)

In [ ]:
plotDensity(mass_accuracy, directory = technical_analysis_directory)

In [ ]:
plotScatter(mass_accuracy, directory = technical_analysis_directory)

### Ion Injection Time

In [ ]:
source("../classes/ion_inject_time.R")

ion_inject_time <- new("IonInjectTime", ms2_scans = data@lfqSequestHT_ms2)

In [ ]:
plot(ion_inject_time, directory = technical_analysis_directory)

### AGC Fill State

In [ ]:
source("../classes/agc_fill.R")

agc_fill <- new("AgcFill", raw_files_path_list = data@lfqRawDataPaths, ms2_scans = data@lfqSequestHT_ms2)

In [ ]:
plot(agc_fill, directory = technical_analysis_directory)

### Missing Data

In [ ]:
source("../classes/missing_data.R")

missing_data_sequest <- new("MissingData", data_frame = data@lfqSequestHT_psms)
missing_data_chimerys <- new("MissingData", data_frame = data@lfqChimerys_psms)

In [ ]:
plotByFile(missing_data_sequest, directory = technical_analysis_directory, node_name = "Sequest HT", node_slug = "sequest")
round(summary(missing_data_sequest@missingValues_file_df$missing_values_rel), 1)

In [ ]:
plotByFile(missing_data_chimerys, directory = technical_analysis_directory, node_name = "CHIMERYS", node_slug = "chimerys")
round(summary(missing_data_chimerys@missingValues_file_df$missing_values_rel), 1)

### Coefficient of Variance

In [ ]:
source("../classes/coefficient_of_variance.R")

protein_cv_sequest <- new("CoefficientOfVariance", protein_df = data@lfqSequestHT_proteins_complete)
protein_cv_chimerys <- new("CoefficientOfVariance", protein_df = data@lfqChimerys_proteins_complete)

In [ ]:
plotCV(protein_cv_sequest, directory = technical_analysis_directory, node_name = "Sequest HT", node_slug = "sequest")
round(summary(protein_cv_sequest@protein_df$CV), 1)

In [ ]:
plotCV(protein_cv_chimerys, directory = technical_analysis_directory, node_name = "CHIMERYS", node_slug = "chimerys")
round(summary(protein_cv_chimerys@protein_df$CV), 1)

## Biological Analysis

In [ ]:
source("../classes/treatment_ratios.R")

treatment_ratios_sequest <- new("TreatmentRatios", protein_df = data@lfqSequestHT_proteins_complete, gene_disease_associations = data@geneDiseaseAssociations)
treatment_ratios_chimerys <- new("TreatmentRatios", protein_df = data@lfqChimerys_proteins_complete, gene_disease_associations = data@geneDiseaseAssociations)

In [ ]:
printStats(treatment_ratios_sequest, node_name = "Sequest HT")

In [ ]:
printStats(treatment_ratios_chimerys, node_name = "CHIMERYS")

### Volcano Plots

In [ ]:
plotVolcanos(treatment_ratios_chimerys, directory = biological_analysis_directory, node_name = "Sequest HT", node_slug = "sequest", show_labels = FALSE, match_gene_list = FALSE)
plotVolcanos(treatment_ratios_sequest, directory = biological_analysis_directory, node_name = "Sequest HT", node_slug = "sequest", show_labels = TRUE, match_gene_list = FALSE)
plotVolcanos(treatment_ratios_sequest, directory = biological_analysis_directory, node_name = "Sequest HT", node_slug = "sequest", show_labels = TRUE, match_gene_list = TRUE)

In [ ]:
plotVolcanos(treatment_ratios_chimerys, directory = biological_analysis_directory, node_name = "CHIMERYS", node_slug = "chimerys", show_labels = FALSE, match_gene_list = FALSE)
plotVolcanos(treatment_ratios_chimerys, directory = biological_analysis_directory, node_name = "CHIMERYS", node_slug = "chimerys", show_labels = TRUE, match_gene_list = FALSE)
plotVolcanos(treatment_ratios_chimerys, directory = biological_analysis_directory, node_name = "CHIMERYS", node_slug = "chimerys", show_labels = TRUE, match_gene_list = TRUE)

### Geno Ontology Analysis

In [ ]:
source("../classes/go_analysis_single.R")

In [ ]:
ratios <- unique(treatment_ratios_sequest@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_sequest@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "up"
  ontology <- "BP"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  go_enrichment <- new(
    "GoAnalysisSingle",
    inherit = treatment_ratios_sequest,
    ratio = ratio,
    regulation = regulation,
    ontology = ontology
  )

  if (is.null(go_enrichment@enrichment) || nrow(as.data.frame(go_enrichment@enrichment)) == 0) next

  plotGO(
    go_enrichment,
    directory = biological_analysis_directory,
    ontology = ontology,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "Sequest HT",
    node_slug = "sequest"
  )
}

for (ratio in ratios) {
  df <- treatment_ratios_sequest@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "down"
  ontology <- "BP"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  go_enrichment <- new(
    "GoAnalysisSingle",
    inherit = treatment_ratios_sequest,
    ratio = ratio,
    regulation = regulation,
    ontology = ontology
  )

  if (is.null(go_enrichment@enrichment) || nrow(as.data.frame(go_enrichment@enrichment)) == 0) next

  plotGO(
    go_enrichment,
    directory = biological_analysis_directory,
    ontology = ontology,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "Sequest HT",
    node_slug = "sequest"
  )
}

In [ ]:
ratios <- unique(treatment_ratios_chimerys@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_chimerys@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "up"
  ontology <- "BP"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  go_enrichment <- new(
    "GoAnalysisSingle",
    inherit = treatment_ratios_chimerys,
    ratio = ratio,
    regulation = regulation,
    ontology = ontology
  )

  if (is.null(go_enrichment@enrichment) || nrow(as.data.frame(go_enrichment@enrichment)) == 0) next

  plotGO(
    go_enrichment,
    directory = biological_analysis_directory,
    ontology = ontology,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "CHIMERYS",
    node_slug = "chimerys"
  )
}

for (ratio in ratios) {
  df <- treatment_ratios_chimerys@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "down"
  ontology <- "BP"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  go_enrichment <- new(
    "GoAnalysisSingle",
    inherit = treatment_ratios_chimerys,
    ratio = ratio,
    regulation = regulation,
    ontology = ontology
  )

  if (is.null(go_enrichment@enrichment) || nrow(as.data.frame(go_enrichment@enrichment)) == 0) next

  plotGO(
    go_enrichment,
    directory = biological_analysis_directory,
    ontology = ontology,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "CHIMERYS",
    node_slug = "chimerys"
  )
}

### Kyoto Encyclopedia of Genes and Genomes (KEGG) Pathway Analysis 

In [ ]:
source("../classes/kegg_analysis_single.R")

In [ ]:
ratios <- unique(treatment_ratios_sequest@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_sequest@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "up"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  kegg_enrichment <- new(
    "KeggAnalysis",
    inherit = treatment_ratios_sequest,
    ratio = ratio,
    regulation = regulation
  )

  if (is.null(kegg_enrichment@enrichment) || nrow(as.data.frame(kegg_enrichment@enrichment)) == 0) next

  plotKEGG(
    kegg_enrichment,
    directory = biological_analysis_directory,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "Sequest HT",
    node_slug = "sequest"
  )
}

for (ratio in ratios) {
  df <- treatment_ratios_sequest@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "down"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  kegg_enrichment <- new(
    "KeggAnalysis",
    inherit = treatment_ratios_sequest,
    ratio = ratio,
    regulation = regulation
  )

  if (is.null(kegg_enrichment@enrichment) || nrow(as.data.frame(kegg_enrichment@enrichment)) == 0) next

  plotKEGG(
    kegg_enrichment,
    directory = biological_analysis_directory,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "Sequest HT",
    node_slug = "sequest"
  )
}

In [ ]:
ratios <- unique(treatment_ratios_chimerys@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_chimerys@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "up"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  kegg_enrichment <- new(
    "KeggAnalysis",
    inherit = treatment_ratios_chimerys,
    ratio = ratio,
    regulation = regulation
  )

  if (is.null(kegg_enrichment@enrichment) || nrow(as.data.frame(kegg_enrichment@enrichment)) == 0) next

  plotKEGG(
    kegg_enrichment,
    directory = biological_analysis_directory,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "CHIMERYS",
    node_slug = "chimerys"
  )
}

for (ratio in ratios) {
  df <- treatment_ratios_chimerys@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)
  regulation <- "down"

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  kegg_enrichment <- new(
    "KeggAnalysis",
    inherit = treatment_ratios_chimerys,
    ratio = ratio,
    regulation = regulation
  )

  if (is.null(kegg_enrichment@enrichment) || nrow(as.data.frame(kegg_enrichment@enrichment)) == 0) next

  plotKEGG(
    kegg_enrichment,
    directory = biological_analysis_directory,
    regulation = regulation,
    treatment = treatment,
    control = control,
    node_name = "CHIMERYS",
    node_slug = "chimerys"
  )
}

### DISGENET Analysis

In [ ]:
source("../classes/disgenet_analysis_single.R")

In [ ]:
ratios <- unique(treatment_ratios_sequest@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_sequest@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  disgenet_sequest <- new(
    "DisgenetAnalysisSingle",
    inherit = treatment_ratios_sequest,
    ratio = ratio
  )

  plotDiseaseClasses(disgenet_sequest, directory = biological_analysis_directory, treatment = treatment, control = control)
}

In [ ]:
ratios <- unique(treatment_ratios_chimerys@ratio_df_consistent$Comparison)

for (ratio in ratios) {
  df <- treatment_ratios_chimerys@ratio_df_consistent %>% filter(Comparison == ratio)

  treatment <- unique(df$Treatment)
  control <- unique(df$Control)

  if (treatment %in% c("CTRL", "DMSO")) next
  if (control == "CTRL") next

  disgenet_chimerys <- new(
    "DisgenetAnalysisSingle",
    inherit = treatment_ratios_chimerys,
    ratio = ratio
  )

  plotDiseaseClasses(disgenet_chimerys, directory = biological_analysis_directory, treatment = treatment, control = control)
}

## Session Info

In [ ]:
library(disgenet2r)
sessionInfo()